<H1><center>Segmentation and Clustering Assignment</center></H1>

Installed BeautifulSoup4

In [15]:
pip install BeautifulSoup4

Note: you may need to restart the kernel to use updated packages.


<h2>Imported all the required libraries</h2>

In [16]:
import requests as re
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np

<h2>Got data from wikipedia and extracted the content</h2>

In [17]:
result = re.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = bs(result.content, 'html.parser')

<h2>Extracted text array from table</h2>

In [18]:
rows = soup.find('tbody').select('tr')
textArray = [r.get_text() for r in rows]

<h2>Data refining</h2>

In [19]:
df = pd.DataFrame(textArray)
df = df[0].str.split('\n', expand=True)
df = df.rename(columns=df.iloc[0])
df = df.drop(df.index[0])
df = df[df.Borough != 'Not assigned']
df = df.groupby(['Postcode', 'Borough'], sort = False).agg(','.join)
df.reset_index(inplace = True)
df = df.replace("Not assigned", "Queen's Park")
df.shape

(103, 3)

<h2>Read CSV for GeoSpatial data</h2>

In [20]:
url = "http://cocl.us/Geospatial_data"
geo_df = pd.read_csv(url)
geo_df.rename(columns={'Postal Code': 'Postcode'}, inplace=True)
df = pd.merge(df, geo_df, on='Postcode')

<h2>Extracted records having toronto in Borough</h2>

In [21]:
toronto_df = df[df['Borough'].str.contains('Toronto')]

In [22]:
toronto_df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
9,M5B,Downtown Toronto,"Ryerson,Garden District",43.657162,-79.378937
15,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
19,M4E,East Toronto,The Beaches,43.676357,-79.293031
20,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [23]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.10

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



<h2>Installed and Imported folium</h2>

In [24]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

<h2>Imported folium and showed map clusters</h2>

In [25]:
address = 'Toronto'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], 
                                           toronto_df['Borough'], toronto_df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)  
    
Toronto_map